In [1]:
import tensorflow as tf
import os

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

In [3]:
daily_sales_numbers

[21, 22, -108, 31, -1, 32, 34, 31]

In [4]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [6]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [7]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [8]:
tf_dataset = tf_dataset.filter(lambda x: x>0)

In [9]:
tf_dataset = tf_dataset.map(lambda x:x*250)

In [10]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

5250
5500
7750
8000
8500
7750


In [11]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

5500
5250
8000
7750
8500
7750


In [12]:
for batch_size in tf_dataset.batch(2):
    print(batch_size.numpy())

[5500 5250]
[7750 8500]
[7750 8000]


### Perform all of the above operations in one sho

In [13]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda y:y*250).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[5250 7750]
[8000 5500]
[8500 7750]


### Images

In [14]:
images_ds = tf.data.Dataset.list_files('../images/*/*', shuffle=False)

In [15]:
image_count = len(images_ds)
image_count

150

In [16]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [17]:
for file in images_ds.take(3):
    print(file.numpy())

b'..\\images\\daisy\\102841525_bd6628ae3c.jpg'
b'..\\images\\daisy\\1031799732_e7f4008c03.jpg'
b'..\\images\\daisy\\105806915_a9c13e2106_n.jpg'


In [18]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'..\\images\\daisy\\512177035_70afc925c8.jpg'
b'..\\images\\roses\\2607130050_9c34310004.jpg'
b'..\\images\\roses\\2409069862_b128ee2a71.jpg'


In [19]:
class_names = ["daisy", "roses"]

In [20]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [21]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [22]:
len(test_ds), len(test_ds)

(30, 30)

In [23]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [24]:
get_label("..\\images\\daisy\\102841525_bd6628ae3c.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'daisy'>

In [25]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [26]:
img, label = process_image("..\\images\\daisy\\102841525_bd6628ae3c.jpg")
img.numpy()[:2]

array([[[1.49506836e+02, 1.41484375e+02, 1.08031250e+02],
        [1.82277344e+02, 1.82669922e+02, 1.61777344e+02],
        [1.98469727e+02, 2.01407227e+02, 1.84594727e+02],
        [1.93666992e+02, 1.97292969e+02, 1.74338867e+02],
        [1.97240234e+02, 1.97208008e+02, 1.63447266e+02],
        [2.17469727e+02, 2.14830078e+02, 1.80173828e+02],
        [2.23830078e+02, 2.23142578e+02, 1.93580078e+02],
        [2.14768555e+02, 2.16021484e+02, 1.97671875e+02],
        [1.98112305e+02, 2.01518555e+02, 1.86877930e+02],
        [1.65562500e+02, 1.68486328e+02, 1.57600586e+02],
        [1.12480469e+02, 1.08995117e+02, 9.88730469e+01],
        [7.05966797e+01, 6.64765625e+01, 5.98935547e+01],
        [8.04726562e+01, 8.03466797e+01, 7.10039062e+01],
        [8.39794922e+01, 8.49648438e+01, 7.88398438e+01],
        [6.76269531e+01, 6.66406250e+01, 6.22519531e+01],
        [4.44062500e+01, 4.51123047e+01, 3.89619141e+01],
        [3.73681641e+01, 3.92060547e+01, 2.84404297e+01],
        [3.231

In [27]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [28]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[2.1914062e+01 1.5342773e+01 3.6425781e-01]
  [2.0707031e+01 1.4250000e+01 0.0000000e+00]
  [2.3114258e+01 1.7200195e+01 0.0000000e+00]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

 [[1.9242188e+01 1.5128906e+01 0.0000000e+00]
  [1.9213867e+01 1.5100586e+01 0.0000000e+00]
  [2.1334961e+01 1.7592773e+01 0.0000000e+00]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

 [[2.1108398e+01 1.5963867e+01 1.7871094e-01]
  [2.0179688e+01 1.5035156e+01 0.0000000e+00]
  [2.1179688e+01 1.6750000e+01 0.0000000e+00]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

 ...

 [[2.0235352e+02 2.1549805e+02 2.1878320e+02]
  [2.0049805e+02 2.1467676e+02 2.1710645e+02]
  [2.0182129e+02 2.1696387e+02 2.1

In [29]:
def scale(image, label):
    return image/255, label

In [30]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[118.796875 125.5625   117.75    ]
  [148.6875   149.4375   141.4375  ]
  [166.625    166.625    157.75    ]
  ...
  [  6.1875     7.1875     1.1875  ]
  [  5.859375   6.859375   0.859375]
  [  4.1875     5.1875     0.      ]]

 [[123.765625 129.34375  121.46875 ]
  [154.6875   155.67188  146.3125  ]
  [176.45312  175.98438  163.75    ]
  ...
  [  7.5        8.5        2.5     ]
  [  5.6875     6.6875     0.6875  ]
  [  5.         6.         0.3125  ]]

 [[122.640625 125.828125 114.265625]
  [151.20312  152.79688  139.8125  ]
  [174.67188  173.       159.9375  ]
  ...
  [  7.9375     8.9375     1.4375  ]
  [  5.4375     6.4375     0.578125]
  [  5.         6.         1.      ]]

 ...

 [[ 62.78125   40.609375  13.625   ]
  [ 60.328125  40.1875    13.1875  ]
  [ 56.0625    38.5       12.59375 ]
  ...
  [ 33.734375  20.71875    5.796875]
  [ 25.578125  15.453125   4.8125  ]
  [ 17.484375  11.703125   2.09375 ]]

 [[ 60.5625    40.25      13.25    ]
  [ 56.78125   39.078

In [31]:
train_ds = train_ds.map(scale)

In [32]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.64150965 0.58679533 0.27266392]
****Label:  b'roses'
****Image:  [4.877329e-01 2.187452e-03 3.401214e-04]
****Label:  b'roses'
****Image:  [0.04720962 0.06666667 0.03921569]
****Label:  b'daisy'
****Image:  [0.9900122 0.9900122 0.9900122]
****Label:  b'roses'
****Image:  [0.3028014  0.37881243 0.16709368]
****Label:  b'daisy'
